### 1 - Importing the test Data

In [9]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2022-11-9", end="2023-01-5", interval='15m')
dataF.iloc[:,:]

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-11-09 00:00:00,1.007861,1.008166,1.007049,1.007049,1.007049,0
2022-11-09 00:15:00,1.007049,1.007455,1.006745,1.007151,1.007151,0
2022-11-09 00:30:00,1.007455,1.007760,1.007252,1.007252,1.007252,0
2022-11-09 00:45:00,1.007049,1.007049,1.006441,1.006644,1.006644,0
2022-11-09 01:00:00,1.006543,1.007049,1.006340,1.006745,1.006745,0
...,...,...,...,...,...,...
2023-01-04 22:45:00,1.060670,1.061121,1.060670,1.060895,1.060895,0
2023-01-04 23:00:00,1.061008,1.061121,1.060558,1.060783,1.060783,0
2023-01-04 23:15:00,1.060895,1.060895,1.060670,1.060895,1.060895,0


### 2- Defining the signal function

In [13]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]

    # Bearish Pattern
    if (open > close and 
        previous_open < previous_close and
        close < previous_open and
        open >= previous_close):
        return 1

    # Bullish Pattern
    elif (open < close and
        previous_open > previous_close and
        close > previous_open and
        open <= previous_close):
        return 2

    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1, len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
# signal_generator(data)
dataF["signal"] = signal


In [14]:
dataF.signal.value_counts()

0    3543
1     176
2     168
Name: signal, dtype: int64

### 3- Connecting to a market and excecuting trades

In [ ]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails 